In [137]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
from tqdm import tqdm

warnings.simplefilter(action="ignore")


def filter_ones(data):
    return data.WEFAC == 1


def filter_more_than_zero(data):
    return data.WEFAC > 0


def filter_all(data):
    return data.WEFAC >= 0


def filter_zeros(data):
    return data.WEFAC == 0


def draw_scaled(data, well):
    columns = ["WWPR", "WOPR", "WGPR", "WBHP", "WTHP", "WGIR", "WWIR"]

    filters = [filter_ones, filter_more_than_zero, filter_all, filter_zeros]
    filter_names = ["WEFAC==1", "WEFAC>0", "WEFAC>=0", "WEFAC==0"]
    sns.set(rc={"figure.figsize": (12, 30)})
    sns.set_theme(style="whitegrid")
    fig, axs = plt.subplots(nrows=4, ncols=1)
    for i, (filt, filt_name) in enumerate(zip(filters, filter_names)):
        cur_data = data[(data.WELL == well) & filt(data)].fillna(0)
        for c in columns:
            cur_data[c] = cur_data[c] / cur_data[columns].max()[c]
        cur_data = cur_data.fillna(0).reset_index()[columns + ["DATE"]]

        for c in columns:
            ax = sns.scatterplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[i])
            sns.lineplot(cur_data["DATE"], cur_data[c], label=c, ax=axs[i])
        plt.legend()
        ax.set(xlabel=f"DATE", ylabel=f"Scaled params")
        ax.title.set_text(filt_name)
    plt.savefig(f"images/merged_scaled_{well}.jpg")
    plt.clf()

In [117]:
data = pd.read_excel("./data/Исходник_для_аномалий.xlsx", decimal='.')
columns = ["WWPR", "WOPR", "WGPR", "WBHP", "WTHP", "WGIR", "WWIR"]
for c in columns+['WEFAC']:
    if data[c].dtype=='O':
        data[c] = pd.Series(data[c].str.replace(',', '.').astype('float64'))
data

,WELL,DATE,WWPR,WOPR,WGPR,WBHP,WTHP,WGIR,WWIR,WEFAC
0,1,2017-09-12,0.0,0.0,NaN,0.0,139.16,0.0,0,1.0
1,1,2017-09-13,0.0,0.0,NaN,0.0,138.18,0.0,0,1.0
2,1,2017-09-14,0.0,0.0,NaN,0.0,138.18,0.0,0,1.0
3,1,2017-09-15,0.0,0.0,NaN,0.0,136.22,0.0,0,1.0
4,1,2017-09-16,0.0,0.0,NaN,0.0,136.22,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
144338,134,2021-12-27,0.0,0.0,NaN,0.0,0.00,0.0,0,1.0
144339,134,2021-12-28,0.0,0.0,NaN,0.0,0.00,0.0,0,1.0
144340,134,2021-12-29,0.0,0.0,NaN,0.0,0.00,0.0,0,1.0
144341,134,2021-12-30,0.0,0.0,NaN,0.0,118.00,0.0,0,1.0


In [ ]:
for well in tqdm(np.unique(data.WELL)):
    draw_scaled(data, well)

  0%|                                                                            | 0/134 [00:00<?, ?it/s]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  1%|▌                                                                   | 1/134 [00:02<05:25,  2.44s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No ar

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
 11%|███████▌                                                           | 15/134 [00:35<04:39,  2.35s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
 12%|████████                                                           | 16/134 [00:37<04:30,  2.30s/it]No ar

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
 22%|██████████████▍                                                    | 29/134 [01:10<04:41,  2.68s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
 22%|███████████████                                                    | 30/134 [01:13<04:26,  2.56s/it]No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No ar